In [6]:
import numpy as np

def feature_sign_search(A, x, y, gamma):
	threshold = 0.00001
	active_set = set()
	t = np.zeros(x.shape)
	
	A2 = np.dot(A.T, A)
	Ay = np.dot(A.T, y)

	while(True):

		grad = - 2 * Ay + 2 * np.dot(A2, x)
		
		condition_a = 0
		if len(grad[t != 0]) != 0:
			condition_a = np.max(abs(grad[t != 0] + gamma * t[t != 0]))
		if len(grad[t == 0]) == 0:
			break
		condition_b = np.max(abs(grad[t == 0]))
		if condition_b < gamma :
			break
			
		if abs(condition_a) < threshold:
			x_max = np.argmax(np.abs(grad) * (t == 0))
			if grad[x_max] > gamma:
				t[x_max] = -1.
			elif grad[x_max] < gamma:
				t[x_max] = 1.
			x[x_max] = 0.
			active_set.add(x_max)
			
		sub = np.array(sorted(active_set))
		sub_matrix = A2[np.ix_(sub, sub)]
		sub_y = Ay[sub]
		sub_t = t[sub]
		lls = sub_y - gamma * sub_t / 2
		new_x = np.linalg.solve(np.atleast_2d(sub_matrix), lls)
		new_t = np.sign(new_x)
		sub_x = x[sub]
		flips = np.where(abs(new_t - sub_t) > 1)[0]
		
		if len(flips) > 0:
			pass                      # not implemented line search!!
		else:
			current = new_x
			
		x[sub] = current
		zeros = sub[np.abs(x[sub]) < 0.000001]
		x[zeros] = 0.
		t[sub] = np.int8(np.sign(x[sub]))
		
		active_set.difference_update(zeros)
	return x


In [7]:
x = np.zeros(6)

y = np.array([2,3,6,4,5])
A = np.array([[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1],[1,0,0,0,0],[0,0,0,0,5]]).T
gamma = 0.000001
x = feature_sign_search(A, x, y, gamma)
print("input:")
print(y)
print("output:")
print(np.matmul(A,x))  
print("selections:")
print(x) 

input:
[2 3 6 4 5]
output:
[ 1.9999995  2.9999995  5.9999995  3.9999995  4.9999999]
selections:
[ 2.9999995   5.9999995   3.9999995   0.          1.9999995   0.99999998]
